# 15 min 2 days

# Range more than 7, for 1h time frame back candle is 32
# Range more than 55 for time frame 1d back candle is 32
# Range more than 4 for time frame 15m back candle is 50

In [1]:
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import linregress
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Function to fetch historical stock data
def fetch_stock_data(ticker_symbol, day_range, time_interval):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=day_range*7//5)
    data = yf.download(ticker_symbol, start=start_date, end=end_date, interval=time_interval)
    return data.dropna()

# Function to preprocess stock data
def preprocess_stock_data(data):
    df_ticker = data[['Open', 'High', 'Low', 'Close']].reset_index()
    df_ticker.columns = ['time', 'open', 'high', 'low', 'close']
    df_ticker = df_ticker.dropna()
    df_ticker.reset_index(drop=True, inplace=True)
    # Add 'pivot' and 'pointpos' columns to the DataFrame
    df_ticker['pivot'] = df_ticker.apply(lambda x: pivot_id(df_ticker, x.name, 3, 3), axis=1)
    df_ticker['pointpos'] = df_ticker.apply(lambda row: point_position(row), axis=1)
    return df_ticker

# Function to calculate pivot points for recent candles
def pivot_id(df1, l, n1, n2):
    # Check if there are enough recent candles to calculate pivot points
    if l - n1 < 0 or l + n2 >= len(df1):
        return 0
    # Initialize pivot flags
    piv_id_low = 1
    piv_id_high = 1
    # Loop over recent candles to check for pivot points
    for i in range(l - n1, l + n2 + 1):
        # Check for low pivot
        if df1.low[l] > df1.low[i]:
            piv_id_low = 0
        # Check for high pivot
        if df1.high[l] < df1.high[i]:
            piv_id_high = 0
    # Determine pivot type
    if piv_id_low and piv_id_high:
        return 3  # Both low and high pivot
    elif piv_id_low:
        return 1  # Low pivot
    elif piv_id_high:
        return 2  # High pivot
    else:
        return 0  # No pivot

# Function to calculate the position of pivot points for recent candles
def point_position(x):
    # Calculate position based on pivot type
    if x['pivot'] == 1:
        return x['low'] - 1e-3  # Low pivot
    elif x['pivot'] == 2:
        return x['high'] + 1e-3  # High pivot
    else:
        return np.nan  # No pivot


def calculate_linear_regression(df_ticker, back_candles, test_size=0.2):
    xxmin = np.array([])
    minim = np.array([])
    xxmax = np.array([])
    high = np.array([])

    for i in range(max(0, len(df_ticker) - back_candles), len(df_ticker)):
        if df_ticker.iloc[i].pivot == 1:
            minim = np.append(minim, df_ticker.iloc[i].low)
            xxmin = np.append(xxmin, i)
        if df_ticker.iloc[i].pivot == 2:
            xxmax = np.append(xxmax, i)
            high = np.append(high, df_ticker.iloc[i].high)

    # Check if there are enough samples available for splitting
    if len(xxmin) < 2 or len(xxmax) < 2:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None

    xxmin_train, xxmin_test, minim_train, minim_test = train_test_split(xxmin, minim, test_size=test_size, random_state=0)
    xxmax_train, xxmax_test, high_train, high_test = train_test_split(xxmax, high, test_size=test_size, random_state=0)

    # Initialize Linear Regression models
    model_min = LinearRegression()
    model_max = LinearRegression()

    # Fit linear regression for low pivot points
    xxmin_train_reshaped = xxmin_train.reshape(-1, 1)
    model_min.fit(xxmin_train_reshaped, minim_train)
    slmin = model_min.coef_[0]
    intercmin = model_min.intercept_

    # Fit linear regression for high pivot points
    xxmax_train_reshaped = xxmax_train.reshape(-1, 1)
    model_max.fit(xxmax_train_reshaped, high_train)
    slmax = model_max.coef_[0]
    intercmax = model_max.intercept_

    # Calculate R-squared for training data
    r_squared_min_train = model_min.score(xxmin_train_reshaped, minim_train)
    r_squared_max_train = model_max.score(xxmax_train_reshaped, high_train)

    # Calculate R-squared for testing data
    r_squared_min_test = model_min.score(xxmin_test.reshape(-1, 1), minim_test)
    r_squared_max_test = model_max.score(xxmax_test.reshape(-1, 1), high_test)

    return xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax, r_squared_min_train, r_squared_max_train, r_squared_min_test, r_squared_max_test


# Function to display candlestick chart with trend prediction
def display_candlestick_chart(df_ticker, back_candles, ticker_symbol, xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax, r_squared_min_train, r_squared_max_train, r_squared_min_test, r_squared_max_test):
    recent_candles = 10  # Number of recent candles to display lines

    df_pl = df_ticker[-(back_candles + recent_candles):]

    # Calculate lines for high and low triangles using training data
    if len(xxmin_train) > 0:
        low_triangle_line_train = slmin * xxmin_train + intercmin
    else:
        low_triangle_line_train = []

    if len(xxmax_train) > 0:
        high_triangle_line_train = slmax * xxmax_train + intercmax
    else:
        high_triangle_line_train = []

    # Predict trend using training data
#     trend_prediction_train = predict_trend(slmin, slmax)

    # Calculate lines for high and low triangles using testing data
    if len(xxmin_test) > 0:
        low_triangle_line_test = slmin * xxmin_test + intercmin
    else:
        low_triangle_line_test = []

    if len(xxmax_test) > 0:
        high_triangle_line_test = slmax * xxmax_test + intercmax
    else:
        high_triangle_line_test = []

    # Predict trend using testing data
#     trend_prediction_test = predict_trend(slmin, slmax)

    # Print R-squared values
    print(f"R-squared (Train) - Low: {r_squared_min_train:.2f}, High: {r_squared_max_train:.2f}")
    print(f"R-squared (Test) - Low: {r_squared_min_test:.2f}, High: {r_squared_max_test:.2f}")

    # Display candlestick chart with pivot points, linear regression lines, and trend prediction
    fig = go.Figure(data=[
        go.Candlestick(x=df_pl.index, open=df_pl['open'], high=df_pl['high'], low=df_pl['low'], close=df_pl['close']),
        go.Scatter(x=df_pl.index, y=df_pl['pointpos'], mode="markers", marker=dict(size=7, color="MediumPurple"), name="pivot"),
        go.Scatter(x=np.append(xxmin_train, xxmin_test), y=np.append(low_triangle_line_train, low_triangle_line_test), mode='lines', line=dict(color='green', width=2)),
        go.Scatter(x=np.append(xxmax_train, xxmax_test), y=np.append(high_triangle_line_train, high_triangle_line_test), mode='lines', line=dict(color='red', width=2))
    ])

    fig.update_layout(
    xaxis_rangeslider_visible=True,
    title=f'Candlestick Chart for {ticker_symbol} ',
    width=900,  # specify width in pixels
    height=600   # specify height in pixels
    )

    fig.show()

# User input for day range and time interval
day_range = int(input("Enter the number of days for historical data: "))
time_interval = input("Enter the time interval (Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]): ")

# Adjust back_candles based on day range and time interval
if time_interval == '1h':
    if day_range > 7:
        back_candles = 32
    else:
        back_candles = 50
elif time_interval == '1d':
    if day_range > 40:
        back_candles = 32
    else:
        back_candles = 32
elif time_interval == '15m':
    if day_range > 4:
        back_candles = 50
    else:
        back_candles = 50
elif time_interval == '5m' or '2m' or '1m':
    if day_range > 2:
        back_candles = 55
    else:
        back_candles = 55
elif time_interval ==  '2m' or '1m':
    if day_range > 2:
        back_candles = 60
    else:
        back_candles = 60
# Ticker symbols to analyze
#ticker_symbols = ["^BSESN","^CNXIT", "^NSEBANK","^NSEI","^CNXFMCG","SBIN.NS","HDFCBANK.NS", "KOTAKBANK.NS","TCS.NS","INFY.NS","ITC.NS","LT.NS"]
ticker_symbols = ["^BSESN","^CNXIT",  "^NSEBANK", "^NSEI", "^CNXFMCG", "^CNXAUTO", "SBIN.NS", "HDFCBANK.NS", "KOTAKBANK.NS", "AXISBANK.NS", "TCS.NS", "INFY.NS", "ITC.NS","LT.NS","HEROMOTOCO.NS","M&M.NS","TVSMOTOR.NS","BAJAJFINSV.NS","ABCAPITAL.NS","ADANIENT.NS","ADANIPORTS.NS"]


# Loop through each ticker symbol

for ticker_symbol in ticker_symbols:
    # Fetch historical data from Yahoo Finance
    data = fetch_stock_data(ticker_symbol, day_range, time_interval)

    # Preprocess stock data
    df_ticker = preprocess_stock_data(data)

    # Calculate linear regression for pivot points
    xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax, r_squared_min_train, r_squared_max_train, r_squared_min_test, r_squared_max_test = calculate_linear_regression(df_ticker, back_candles)

    xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax, r_squared_min_train, r_squared_max_train, r_squared_min_test, r_squared_max_test = calculate_linear_regression(df_ticker, back_candles)

# Check if any of the returned variables are None
    if any(var is None for var in [xxmin_train, xxmax_train]):
        print(f"Skipping {ticker_symbol} due to insufficient data for linear regression.")
        continue

    # Check if training set has sufficient samples for splitting
    if len(xxmin_train) < 2 or len(xxmax_train) < 2:
        print(f"Skipping {ticker_symbol} due to insufficient data for linear regression.")
        continue


    # Display candlestick chart with pivot points, linear regression lines, and trend prediction for training data
    display_candlestick_chart(df_ticker, back_candles, ticker_symbol, xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax, r_squared_min_train, r_squared_max_train, r_squared_min_test, r_squared_max_test)


ModuleNotFoundError: No module named 'yfinance'

In [9]:
back_candles

50

In [1]:
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import linregress
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Function to fetch historical stock data
def fetch_stock_data(ticker_symbol, day_range, time_interval):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=day_range*7//5)
    data = yf.download(ticker_symbol, start=start_date, end=end_date, interval=time_interval)
    return data.dropna()

# Function to preprocess stock data
def preprocess_stock_data(data):
    df_ticker = data[['Open', 'High', 'Low', 'Close']].reset_index()
    df_ticker.columns = ['time', 'open', 'high', 'low', 'close']
    df_ticker = df_ticker.dropna()
    df_ticker.reset_index(drop=True, inplace=True)
    # Add 'pivot' and 'pointpos' columns to the DataFrame
    df_ticker['pivot'] = df_ticker.apply(lambda x: pivot_id(df_ticker, x.name, 3, 3), axis=1)
    df_ticker['pointpos'] = df_ticker.apply(lambda row: point_position(row), axis=1)
    return df_ticker

# Function to calculate pivot points for recent candles
def pivot_id(df1, l, n1, n2):
    # Check if there are enough recent candles to calculate pivot points
    if l - n1 < 0 or l + n2 >= len(df1):
        return 0
    # Initialize pivot flags
    piv_id_low = 1
    piv_id_high = 1
    # Loop over recent candles to check for pivot points
    for i in range(l - n1, l + n2 + 1):
        # Check for low pivot
        if df1.low[l] > df1.low[i]:
            piv_id_low = 0
        # Check for high pivot
        if df1.high[l] < df1.high[i]:
            piv_id_high = 0
    # Determine pivot type
    if piv_id_low and piv_id_high:
        return 3  # Both low and high pivot
    elif piv_id_low:
        return 1  # Low pivot
    elif piv_id_high:
        return 2  # High pivot
    else:
        return 0  # No pivot

# Function to calculate the position of pivot points for recent candles
def point_position(x):
    # Calculate position based on pivot type
    if x['pivot'] == 1:
        return x['low'] - 1e-3  # Low pivot
    elif x['pivot'] == 2:
        return x['high'] + 1e-3  # High pivot
    else:
        return np.nan  # No pivot


def calculate_linear_regression(df_ticker, back_candles, test_size=0.2):
    xxmin = np.array([])
    minim = np.array([])
    xxmax = np.array([])
    high = np.array([])

    for i in range(max(0, len(df_ticker) - back_candles), len(df_ticker)):
        if df_ticker.iloc[i].pivot == 1:
            minim = np.append(minim, df_ticker.iloc[i].low)
            xxmin = np.append(xxmin, i)
        if df_ticker.iloc[i].pivot == 2:
            xxmax = np.append(xxmax, i)
            high = np.append(high, df_ticker.iloc[i].high)

    # Check if there are enough samples available for splitting
    if len(xxmin) < 2 or len(xxmax) < 2:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None

    xxmin_train, xxmin_test, minim_train, minim_test = train_test_split(xxmin, minim, test_size=test_size, random_state=0)
    xxmax_train, xxmax_test, high_train, high_test = train_test_split(xxmax, high, test_size=test_size, random_state=0)

    # Initialize Linear Regression models
    model_min = LinearRegression()
    model_max = LinearRegression()

    # Fit linear regression for low pivot points
    xxmin_train_reshaped = xxmin_train.reshape(-1, 1)
    model_min.fit(xxmin_train_reshaped, minim_train)
    slmin = model_min.coef_[0]
    intercmin = model_min.intercept_

    # Fit linear regression for high pivot points
    xxmax_train_reshaped = xxmax_train.reshape(-1, 1)
    model_max.fit(xxmax_train_reshaped, high_train)
    slmax = model_max.coef_[0]
    intercmax = model_max.intercept_

    return xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax


# Function to display candlestick chart with trend prediction
def display_candlestick_chart(df_ticker, back_candles, ticker_symbol, xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax):
    recent_candles = 10  # Number of recent candles to display lines

    df_pl = df_ticker[-(back_candles + recent_candles):]

    # Calculate lines for high and low triangles using training data
    if len(xxmin_train) > 0:
        low_triangle_line_train = slmin * xxmin_train + intercmin
    else:
        low_triangle_line_train = []

    if len(xxmax_train) > 0:
        high_triangle_line_train = slmax * xxmax_train + intercmax
    else:
        high_triangle_line_train = []

    # Calculate lines for high and low triangles using testing data
    if len(xxmin_test) > 0:
        low_triangle_line_test = slmin * xxmin_test + intercmin
    else:
        low_triangle_line_test = []

    if len(xxmax_test) > 0:
        high_triangle_line_test = slmax * xxmax_test + intercmax
    else:
        high_triangle_line_test = []

    # Display candlestick chart with pivot points, linear regression lines, and trend prediction
    fig = go.Figure(data=[
        go.Candlestick(x=df_pl.index, open=df_pl['open'], high=df_pl['high'], low=df_pl['low'], close=df_pl['close']),
        go.Scatter(x=df_pl.index, y=df_pl['pointpos'], mode="markers", marker=dict(size=7, color="MediumPurple"), name="pivot"),
        go.Scatter(x=np.append(xxmin_train, xxmin_test), y=np.append(low_triangle_line_train, low_triangle_line_test), mode='lines', line=dict(color='green', width=2)),
        go.Scatter(x=np.append(xxmax_train, xxmax_test), y=np.append(high_triangle_line_train, high_triangle_line_test), mode='lines', line=dict(color='red', width=2))
    ])

    fig.update_layout(
    xaxis_rangeslider_visible=True,
    title=f'Candlestick Chart for {ticker_symbol} ',
    width=900,  # specify width in pixels
    height=600   # specify height in pixels
    )

    fig.show()

# User input for day range and time interval
day_range = int(input("Enter the number of days for historical data: "))
time_interval = input("Enter the time interval (Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]): ")

# Adjust back_candles based on day range and time interval
if time_interval == '1h':
    if day_range > 7:
        back_candles = 32
    else:
        back_candles = 50
elif time_interval == '1d':
    if day_range > 40:
        back_candles = 32
    else:
        back_candles = 32
elif time_interval == '15m':
    if day_range > 4:
        back_candles = 50
    else:
        back_candles = 50
elif time_interval == '5m' or '2m' or '1m':
    if day_range > 2:
        back_candles = 55
    else:
        back_candles = 55
elif time_interval ==  '2m' or '1m':
    if day_range > 2:
        back_candles = 60
    else:
        back_candles = 60
# Ticker symbols to analyze
#ticker_symbols = ["^BSESN","^CNXIT", "^NSEBANK","^NSEI","^CNXFMCG","SBIN.NS","HDFCBANK.NS", "KOTAKBANK.NS","TCS.NS","INFY.NS","ITC.NS","LT.NS"]
ticker_symbols = ["^BSESN","^CNXIT",  "^NSEBANK", "^NSEI", "^CNXFMCG", "^CNXAUTO", "SBIN.NS", "HDFCBANK.NS", "KOTAKBANK.NS", "AXISBANK.NS", "TCS.NS", "INFY.NS", "ITC.NS","LT.NS","HEROMOTOCO.NS","M&M.NS","TVSMOTOR.NS","BAJAJFINSV.NS","ABCAPITAL.NS","ADANIENT.NS","ADANIPORTS.NS"]


# Loop through each ticker symbol
for ticker_symbol in ticker_symbols:
    # Fetch historical data from Yahoo Finance
    data = fetch_stock_data(ticker_symbol, day_range, time_interval)

    # Preprocess stock data
    df_ticker = preprocess_stock_data(data)

    # Calculate linear regression for pivot points
    xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax = calculate_linear_regression(df_ticker, back_candles)

    # Check if any of the returned variables are None
    if any(var is None for var in [xxmin_train, xxmax_train]):
        print(f"Skipping {ticker_symbol} due to insufficient data for linear regression.")
        continue

    # Display candlestick chart with pivot points and linear regression lines
    display_candlestick_chart(df_ticker, back_candles, ticker_symbol, xxmin_train, xxmin_test, minim_train, minim_test, xxmax_train, xxmax_test, high_train, high_test, slmin, intercmin, slmax, intercmax)


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


ValueError: too many values to unpack (expected 12)